In [1]:
# Load Libraries
import pandas as pd
import numpy as np
import re, os
from string import printable
from sklearn import model_selection

import tensorflow as tf
from keras.models import Sequential, Model, model_from_json, load_model
from keras import regularizers
from keras.layers.core import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Input, ELU, LSTM, Embedding, Convolution2D, MaxPooling2D, \
    BatchNormalization, Convolution1D, MaxPooling1D, concatenate
from keras.preprocessing import sequence
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras import backend as K
from sklearn.model_selection import KFold
from pathlib import Path
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
from keras.utils import plot_model
from tensorflow.python.platform import gfile
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, Ridge, Lasso
from keras.utils import to_categorical
from numpy import dstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from numpy import argmax
from keras.models import model_from_json

from model_preprocessor import Preprocessor

import json

import warnings
warnings.filterwarnings("ignore")
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
K.tensorflow_backend.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
# General save model to disk function
def save_model(fileModelJSON, fileWeights):
    if Path(fileModelJSON).is_file():
        os.remove(fileModelJSON)
    json_string = stacked_model.to_json()
    with open(fileModelJSON, 'w') as f:
        json.dump(json_string, f)
    if Path(fileWeights).is_file():
        os.remove(fileWeights)
    stacked_model.save_weights(fileWeights)

def load_models(fileModelJSON, fileWeights):
    with open(fileModelJSON, 'r') as f:
        model_json = f.read()
        model = model_from_json(model_json)
        f.close()
    model.load_weights(fileWeights)
    return model

In [3]:
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False
            # rename to avoid 'unique layer name' issue
            layer.name = 'ensemble_' + str(i+1) + '_' + layer.name
    # define multi-headed input
    ensemble_visible = [model.input for model in members]
    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]

    merge = concatenate(ensemble_outputs)
    hidden = Dense(32, activation='relu')(merge)
    output = Dense(21, activation='softmax')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

    # fit a stacked model
def fit_stacked_model(model, inputX, inputy, epochs, batch_size):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]
    # encode output data
    inputy_enc = to_categorical(inputy)

    # fit model
    model.fit(X, inputy, epochs=epochs, batch_size=batch_size)

# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]
    # make prediction
    return model.predict(X, verbose=0)

In [4]:
# Load data using model preprocessor
x_train, x_test, y_train, y_test = Preprocessor.load_data()

In [5]:
models_dir = "saved_model/"
CNNmodel = load_models(models_dir + "cnn.json", models_dir + "cnn.h5")
LSTMmodel = load_models(models_dir + "lstm.json", models_dir + "lstm.h5")
GRUMmodel = load_models(models_dir + "gru.json", models_dir + "gru.h5")

adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

CNNmodel.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
LSTMmodel.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
GRUMmodel.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

members = list()
members.append(CNNmodel)
members.append(LSTMmodel)
members.append(GRUMmodel)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
epochs = 5
batch_size = 32

# define ensemble model
stacked_model = define_stacked_model(members)

print("################################")
print("Complete Define Stacked Model!!") 


# fit stacked model on test dataset
print("#################################")
print("Start training!!!!")

fit_stacked_model(stacked_model, x_train, y_train, epochs, batch_size)
print("Complete Train Stacked Model!!")

################################
Complete Define Stacked Model!!
#################################
Start training!!!!
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
1292096/1292096 [==============================] - 912s 706us/step - loss: 0.2004 - accuracy: 0.9480
Epoch 2/5
1292096/1292096 [==============================] - 904s 700us/step - loss: 0.1802 - accuracy: 0.9503
Epoch 3/5
1292096/1292096 [==============================] - 902s 698us/step - loss: 0.1766 - accuracy: 0.9509
Epoch 4/5
1292096/1292096 [==============================] - 902s 698us/step - loss: 0.1741 - accuracy: 0.9511
Epoch 5/5
1292096/1292096 [==============================] - 899s 696us/step - loss: 0.1729 - accuracy: 0.9514
Complete Train Stacked Model!!


In [9]:
for model in members:
    _, acc = model.evaluate(x_test, y_test, verbose=0)
    print('Model Accuracy: %.3f' % acc)

X = [x_test for _ in range(len(stacked_model.input))]
stacked_model.evaluate(X, y_test)

Model Accuracy: 0.933
Model Accuracy: 0.944
Model Accuracy: 0.947
143567/143567 [==============================] - 97s 679us/step


[0.15821998300758425, 0.955902099609375]

In [10]:
from keras import backend as K

sess = K.get_session()
init = tf.global_variables_initializer()
sess.run(init)

builder = tf.saved_model.builder.SavedModelBuilder("./saved_model/stacking/")
builder.add_meta_graph_and_variables(sess, [tf.saved_model.tag_constants.SERVING])
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/stacking/saved_model.pb


b'./saved_model/stacking/saved_model.pb'